<a href="https://colab.research.google.com/github/veryHapppy/study_ai/blob/main/Kaggle/plant_seedlings_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna
!pip install plotly
!pip install konlpy
!pip install transformers datasets accelerate
!pip install evaluate
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import optuna
import optuna.visualization as vis
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import itertools
from sklearn.metrics import confusion_matrix
from konlpy.tag import Okt
import urllib.request
import pickle
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import evaluate
from transformers import pipeline
import os
from torch.utils.data import Dataset, DataLoader, random_split
import zipfile
import json
import torchvision.models as models
from torchvision.datasets import ImageFolder

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 컬러 이미지에선 필수!
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 2. 테스트/검증용 (증강 제외 - 깨끗한 원본)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 2. 데이터셋 생성 (경로를 본인의 폴더명으로 바꾸세요)

zip_path = '/content/drive/MyDrive/Colab Notebooks/Plant Seedlings Classification/train.zip'
extract_path = '/content/dataset/train_data' # 코랩 내부 경로

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

full_dataset = ImageFolder(root='/content/dataset/train_data', transform=None)


# 2. 비율 정하기 (예: 80%는 학습용, 20%는 검증용)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# 3. 무작위로 데이터 분할
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

class ApplyTransform(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        # DogCatDataset에서 이미 PIL 이미지를 읽어오므로 바로 적용 가능
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# 이제 각각의 로더에 맞게 적용
train_dataset = ApplyTransform(train_dataset, transform=train_transform)
val_dataset = ApplyTransform(val_dataset, transform=test_transform)

# 4. 각각의 DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) # 검증은 섞을 필요 없음

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def train_and_eval(model, train_loader, test_loader, optimizer, criterion):
    # 3 에폭 정도만 학습해서 성능을 확인 (시간 절약)
    for epoch in range(2):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # 최종 검증 단계
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

def objective_step1(trial):
    # 1. 학습 환경 변수만 제안받음
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    # 모델은 기본 구조로 고정
    model = models.resnet18(weights='DEFAULT')
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 12) # 12가지 식물 분류
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    # 학습 (1~2 에폭만 짧게 수행)
    acc = train_and_eval(model, train_loader, val_loader, optimizer, criterion)
    return acc

# 1단계 실행
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective_step1, n_trials=20) # 시도횟수

best_lr = study1.best_params['lr']
best_batch_size = study1.best_params['batch_size']
best_weight_decay = study1.best_params['weight_decay']
print(f"1단계 완료, 최적의 lr: {best_lr}, batch_size: {best_batch_size}, weight_decay: {best_weight_decay}")

[I 2025-12-28 04:27:24,238] A new study created in memory with name: no-name-f5ef029e-14b8-4616-a302-08dc8a93e4c1


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 130MB/s]
[I 2025-12-28 04:29:22,581] Trial 0 finished with value: 93.26315789473684 and parameters: {'lr': 0.00028353853383692004, 'batch_size': 256, 'weight_decay': 1.853470462486221e-05}. Best is trial 0 with value: 93.26315789473684.
[I 2025-12-28 04:31:03,983] Trial 1 finished with value: 15.578947368421053 and parameters: {'lr': 0.00988499655600463, 'batch_size': 128, 'weight_decay': 5.36598792774104e-06}. Best is trial 0 with value: 93.26315789473684.
[I 2025-12-28 04:32:42,797] Trial 2 finished with value: 73.6842105263158 and parameters: {'lr': 0.0017173651766575816, 'batch_size': 256, 'weight_decay': 9.249708702675652e-05}. Best is trial 0 with value: 93.26315789473684.
[I 2025-12-28 04:34:19,231] Trial 3 finished with value: 76.10526315789474 and parameters: {'lr': 0.0012821120437221058, 'batch_size': 128, 'weight_decay': 0.000800019204094495}. Best is trial 0 with value: 93.26315789473684.
[I 2025-12-28 04:35:56,663] Trial 4 finishe

KeyboardInterrupt: 

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Plant Seedlings Classification/best_params_step1.json', 'w') as f:
    json.dump(study1.best_params, f)

In [ ]:
# 2. 그 레시피대로 '진짜 최종 모델'을 생성합니다.
final_model = models.resnet18(weights='DEFAULT')
num_ftrs = final_model.fc.in_features
final_model.fc = nn.Linear(num_ftrs, 12) # 12가지 식물 분류
final_model = final_model.to(device)

optimizer = optim.Adam(final_model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
criterion = nn.CrossEntropyLoss()

print(f"최종 모델 구성 완료")

history = {'train_loss': [], 'val_acc': []}

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset = ApplyTransform(train_dataset, transform=train_transform)
val_dataset = ApplyTransform(val_dataset, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size,
                          shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

epochs = 30
for epoch in range(epochs):
    final_model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # 에폭마다 검증 (테스트셋 활용)
    final_model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = final_model(images)
            _, pred = torch.max(outputs, 1)
            correct += (pred == labels).sum().item()

    acc = 100 * correct / len(val_dataset)
    avg_loss = total_loss / len(train_loader)

    history['train_loss'].append(avg_loss)
    history['val_acc'].append(acc)

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}, Acc: {acc:.2f}%")

print("최종 모델 학습 완료")

최종 모델 구성 완료
Epoch [1/30] - Loss: 0.9567, Acc: 89.68%
Epoch [2/30] - Loss: 0.2208, Acc: 93.58%
Epoch [3/30] - Loss: 0.1464, Acc: 92.21%
Epoch [4/30] - Loss: 0.1043, Acc: 94.32%
Epoch [5/30] - Loss: 0.0625, Acc: 95.47%
Epoch [6/30] - Loss: 0.0583, Acc: 94.21%
Epoch [7/30] - Loss: 0.0536, Acc: 95.26%
Epoch [8/30] - Loss: 0.0388, Acc: 95.58%
Epoch [9/30] - Loss: 0.0246, Acc: 94.32%
Epoch [10/30] - Loss: 0.0315, Acc: 95.05%
Epoch [11/30] - Loss: 0.0337, Acc: 93.79%
Epoch [12/30] - Loss: 0.0198, Acc: 94.42%
Epoch [13/30] - Loss: 0.0142, Acc: 95.68%
Epoch [14/30] - Loss: 0.0158, Acc: 96.00%
Epoch [15/30] - Loss: 0.0110, Acc: 95.47%
Epoch [16/30] - Loss: 0.0126, Acc: 95.47%
Epoch [17/30] - Loss: 0.0084, Acc: 95.47%
Epoch [18/30] - Loss: 0.0305, Acc: 94.53%
Epoch [19/30] - Loss: 0.0267, Acc: 96.11%
Epoch [20/30] - Loss: 0.0361, Acc: 93.68%
Epoch [21/30] - Loss: 0.0376, Acc: 93.89%
Epoch [22/30] - Loss: 0.0269, Acc: 92.53%
Epoch [23/30] - Loss: 0.0215, Acc: 91.89%
Epoch [24/30] - Loss: 0.0128, A

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/Plant Seedlings Classification'

if not os.path.exists(save_path):
    os.makedirs(save_path)

# 모델 저장 (가장 성능이 좋았을 때 실행)
torch.save(final_model.state_dict(), f"{save_path}/final_model.pth")

In [ ]:
class PlantTestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # test 폴더 내의 모든 이미지 파일 목록 (정렬하여 순서 보장)
        self.file_list = sorted([f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = self.file_list[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name # 이미지와 파일명을 함께 반환

def generate_submission(model, test_loader, class_names, save_path):
    model.eval()
    all_preds = []
    all_files = []

    with torch.no_grad():
        for images, file_names in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1) # 가장 높은 확률의 인덱스 추출

            # 인덱스를 클래스 이름으로 변환
            for i in range(len(predicted)):
                all_preds.append(class_names[predicted[i].item()])
                all_files.append(file_names[i])

    # 데이터프레임 생성
    submission = pd.DataFrame({
        'file': all_files,
        'species': all_preds
    })

    submission.to_csv(save_path, index=False)
    print(f"제출 파일이 저장되었습니다: {save_path}")

# --- 실행 부분 ---

# 1. 테스트 데이터 압축 해제 (이미 하셨다면 생략)
test_zip_path = '/content/drive/MyDrive/Colab Notebooks/Plant Seedlings Classification/test.zip'
with zipfile.ZipFile(test_zip_path, 'r') as z:
    z.extractall('/content/dataset/test_data')

# 2. 테스트 로더 설정 (test_transform 사용)
test_dataset = PlantTestDataset(root_dir='/content/dataset/test_data', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. 클래스 이름 목록 가져오기 (ImageFolder에서 썼던 순서 그대로)
class_names = full_dataset.breeds

# 4. 제출 파일 생성
generate_submission(final_model, test_loader, class_names, '/content/drive/MyDrive/Colab Notebooks/Plant Seedlings Classification/plant_submission.csv')

제출 파일이 저장되었습니다: /content/drive/MyDrive/Colab Notebooks/Plant Seedlings Classification/plant_submission.csv
